In [1]:
import apexpy as ap
from datetime import datetime,timedelta
import numpy as np

ModuleNotFoundError: No module named 'apexpy'

In [4]:
#create dictionary for the GRB's containing lon,lat,height and T0
#from dict, convert from geo to apex, then geo to mlat,mlt (magnetic latitude -and longitude)


#entries in d--> 0:lat, 1:lon, 2:height,3:T0, 4:[tshift]
#only a handful of burst I have extracted data from
d = {#'GRB180720':[19.646,105.226,403.5,'2018-07-20T14:21:47.389'],[],
     #'GRB181222':[-34.591,-7.559,413.69,'2018-12-22T20:11:36.576'],[],
     'GRB190206':[35.715,149.173,407.58,'2019-02-06T03:49:28.525',[130]],
     'GRB190305':[-51.772,84.887,429.04,'2019-03-05T13:05:19.816',[-92]],
     'GRB190606':[-22.314,2.778,424.53,'2019-06-06T01:55:06.950',[-1]],
     #'GRB200415':[37.812,136.225,420.88,'2020-04-15T08:48:05.528',[]],
     #'GRB200521':[-49.539,-25.451,435.36,'2020-05-21T12:16:39.818',[]],
     #'GRB200716':[-39.436,-88.367,431.88,'2020-07-16T22:57:40.634',[]],
     'GRB201227':[-42.943,114.359,433.84,'2020-12-27T15:14:06.853',[]],
     #'GRB210424':[-21.355,144.45,424.88,'2021-04-24T08:01:54.997',[]],
     #'GRB210619':[-33.466,59.682,432.16,'2021-06-19T23:59:24.915',[]],
     'GRB211211':[-38.614,141.956,430.77,'2021-12-11T13:10:01.323',[]]
}
#have to shift almost every T0 by tshift. Bkg is happening before or after T0

In [6]:
def get_mlat(l):
    lat,lon,height,T0 = l[0],l[1],l[2],datetime.fromisoformat(l[3])
    tshift = l[4]
    print(T0)
    for s in tshift:
        A = ap.Apex(date=T0+timedelta(seconds=s))
        mlat, mlon = A.convert(lat, lon, 'geo', 'apex', height=height)
        # geo to MLT
        #print(mlat,mlon)
        mlat, mlt = A.convert(lat, lon, 'geo', 'mlt', datetime=T0, height=height)
        print(f'mag lat for tshift = {np.round(s,2)}',np.round(mlat,3))


In [33]:
get_mlat(d['GRB190305'])

2019-03-05 13:05:19.816000
-63.788


In [7]:
for key,value in d.items():
    get_mlat(value)

NameError: name 'datetime' is not defined